<a href="https://colab.research.google.com/github/hbdarby/scratchblocks/blob/master/BFDAI_Ngrok_Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Credit to CookiePPP for almost all of the code!

In [ ]:
# (optional) check graphics card
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-ed58afc7-273d-0e56-433a-0ee688698e5c)


# 1 - Mount Google Drive and add model shortcut
(you can also download/clone the models if you want them local/safe)

1.1 - Go to [This link.](https://drive.google.com/drive/folders/1M0e6qxPmF3FB3rBWCuYIjwIxwHDADpAI?usp=sharing)

1.2 - Go to "Shared with me", and select the "BFDAI Ngrok" folder, but DON'T go into it, only select it.  Press Shift+Z to add the folder to your drive (it will take up a bit of space!)

1.3 - Mount your drive using the codebox below.

1.4 - Run the code block after that and continue if there is no error.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# check the shared folder has been mounted
import os
assert os.path.exists("/content/drive/My Drive/BFDAI Ngrok"), f"BFDAI Ngrok folder not found! Be sure you've mounted your Drive and added the folder!"
print("Assertion Passed! You're good to go.")

Assertion Passed! You're good to go.


# 2 - Download and Install Source code + Dependencies

Run the codebox below.

In [ ]:
%tensorflow_version 1.x
# clone cookie
import os
!git clone https://github.com/RAYTRAC3R/cookietts
!git submodule init
!git submodule update
!pip install -q unidecode tensorboardX emoji

# clone and install nvidia/apex
!git clone -q https://github.com/NVIDIA/apex.git
!cd apex; pip install -q -v --no-cache-dir ./; cd ..

# install Sox (with all optional formats)
!apt -qq install -y libsox-fmt-all sox

# Downgrade Pytorch (new version has reversed order for argmax() func)
!pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

#Install the last of the needed modules!
%cd /content/cookietts
!pip install -e .
!pip install -r requirements.txt
%cd /content/cookietts/CookieTTS/_5_infer/t2s_server

TensorFlow 1.x selected.
Cloning into 'cookietts'...
remote: Enumerating objects: 2281, done.
remote: Counting objects: 100% (253/253), done.
remote: Compressing objects: 100% (157/157), done.
remote: Total 2281 (delta 142), reused 135 (delta 80), pack-reused 2028
Receiving objects: 100% (2281/2281), 223.16 MiB | 30.60 MiB/s, done.
Resolving deltas: 100% (1427/1427), done.
Checking out files: 100% (461/461), done.
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
     |████████████████████████████████| 245kB 14.5MB/s 
     |████████████████████████████████| 122kB 20.0MB/s 
     |████████████████████████████████| 133kB 23.1MB/s 
Processing /content/apex
  Created wheel for apex: filename=apex-0.1-cp37-none-any.whl size=204330 sha256=030c30c2cf9c24a57b3bb38ac1aecd8101b4d12269b2b4c02587eb3e7335e2d0
  Stored in directory: /tmp/pip-ephem-wheel-cache-fv68ise8/wheels/b1/3a/aa/d84906eaab780ae580c7a5686a33bf

# 3 - Update config

The following codebox will point the webapp towards the correct files.

## BFDAI Ngrok Model V1.4

Changelog:

V1.45 (February 10th, 2021) - Added Classic Pen, Nickel, and Pen.  I've also added back version switching, because some characters might be a bit wonky in this new version.

V1.4 (Dec 25th, 2020) - Added Cake, Classic Match, Grassy, Match, Marker, Naily, Taco, and X.  Had to retrain almost entirely from scratch, so quality has gone down again.

V1.31 (Nov 15th, 2020) - More training.  ~~Some characters might sound a bit worse in this version, so I've added the option to switch between this and the last version.~~ Removed the version switching for now, might add it again later.

V1.3 (Nov 12th, 2020) - Added Loser.

V1.2 (Nov 11th, 2020) - Added Bottle, Classic Coiny, Gelatin, Liy, and Lollipop.

V1.1 (Nov 9th, 2020) - Added Leafy in both regular and classic forms.

V1 (Nov 9th, 2020) - First release

In [ ]:
%%writefile t2s_config.json

{
    "terminal": {
        "show_inference_progress": true,
        "show_inference_alignment_scores": true
    },
    "webpage": {
        "max_input_len": "99999999",
        "sort_speakers": true,
        "localhost": false,
        "defaults": {
            "current_text"       : "",
            "background_text"    : "Enter text.",
            "speaker"            : ["Four"],
            "use_arpabet"        : "on",
            "cat_silence_s"      : 0.20,
            "batch_size"         : 256,
            "max_attempts"       : 64,
            "max_duration_s"     : 12.0,
            "dyna_max_duration_s": 0.125,
            "textseg_len_target" : 120,
            "split_nl"           : "on",
            "split_quo"          : "on",
            "multispeaker_mode"  : "random",
            "growl"              : 0.0
        }
    },
    "workers": {
        "devices": ["cpu", "cuda:0"],
        "sox_output_ext": "wav",
        "working_directory": "server_infer",
        "output_directory": "server_infer_done",
        "output_maxsize_gb": 0.01,
        "dict_path": "/content/drive/MyDrive/BFDAI Ngrok/merged.dict.txt",
        "TTM": {
            "default_model": "BFDAI V1.45",
            "models": {
                "BFDAI V1.45": {
                    "modelpath": "/content/drive/MyDrive/BFDAI Ngrok/checkpoint_64000",
                    "speaker_ids_file": "/content/drive/MyDrive/BFDAI Ngrok/speakerlist 1.45.txt",
                    "use_speaker_ids_file_override": true
                },
                "BFDAI V1.4": {
                    "modelpath": "/content/drive/MyDrive/BFDAI Ngrok/checkpoint_46500",
                    "speaker_ids_file": "/content/drive/MyDrive/BFDAI Ngrok/speaker_ids_4.txt",
                    "use_speaker_ids_file_override": true
                }

            }
        },
        "MTW": {
            "default_model": "MEL | HiFi-GAN",
            "models": {
                "MEL | HiFi-GAN": {
                    "modelpath": "/content/drive/MyDrive/BFDAI Ngrok/vocoder"
                }
            }
        }
    }
}

Overwriting t2s_config.json


# 3 - Start server

### [Note: Use the ngrok link!]

[Example Image -> https://u.smutty.horse/lvecaonealc.png](https://u.smutty.horse/lvecaonealc.png)

Once the link is working, it'll last until you run out of time or you stop the codebox, whichever comes first.  You can share the link with other people!

In [ ]:
# quickly check graphics card
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-b5ea5d1d-1fa0-48fc-7f0f-ad950b75f4da)


In [ ]:
# link through google proxy, might have firewall issues
from google.colab.output import eval_js
print("Ignore this link.\n", eval_js("google.colab.kernel.proxyPort(5000)"), "\n")
 
# link through ngrok, bandwidth is questionable
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip
print("\n\nUse this link once the server is connected!")
get_ipython().system_raw('./ngrok http 5000 &')
from time import sleep
sleep(2)
! curl -s http://localhost:4040/api/tunnels | python3 -c \
 "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"
 
print("Starting server. Connect once 'T2S Initialized and Ready!' is shown\n")
#from subprocess import Popen
#p = Popen(['python3', 'app.py'])
!python3 app.py

Streaming output truncated to the last 5000 lines.

Input_Str 03: "VANESSA What happened here?"
Score_Str 03: [0.641weighted_score] [1.083 diagonality] [64.11%avg_max_att] [ 0.00diagonality_punishment] [ 0.00max_dur_punishment] [ 0.00min_dur_punishment] [ 0.00avg_dur_punishment] [ 0.00mis_dur_punishment]|

1115/1272, 1.35mins remaining.
Generated 5125.166s of audio in 577.783s wall time - so far. (best of 256 tries this pass) (8.87xRT) (86.2%Failure Rate)


tensor([13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
        13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
        13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
        13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
        13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
        13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
        13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13